In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [2]:
try:
    project_root = Path.cwd().parent
    features_path = project_root / "data" / "processed" / "features_regresion.csv"
    df_reg = pd.read_csv(features_path)
    print("DataFrame para regresión cargado.")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en: {features_path}")
    df_reg = None

DataFrame para regresión cargado.


In [3]:
X = df_reg.drop(['pay_amt_june', 'ID'], axis=1)
y = df_reg['pay_amt_june']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

In [6]:
print("\n--- Resultados Regresión Lineal ---")
print(f"R^2 Score: {r2_score(y_test, y_pred_lr):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lr)):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_lr):.4f}")



--- Resultados Regresión Lineal ---
R^2 Score: 0.3274
RMSE: 10493.8009
MAE: 4727.8088


In [7]:
huber = HuberRegressor()
huber.fit(X_train_scaled, y_train)
y_pred_huber = huber.predict(X_test_scaled)

In [8]:
print("\n--- Resultados Regresión de Huber ---")
print(f"R^2 Score: {r2_score(y_test, y_pred_huber):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_huber)):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_huber):.4f}")



--- Resultados Regresión de Huber ---
R^2 Score: 0.1639
RMSE: 11699.3685
MAE: 3251.2201


In [9]:
from sklearn.ensemble import RandomForestRegressor

In [10]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [12]:
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_reg.fit(X_train_scaled, y_train_log)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [13]:
y_pred_log_rf = rf_reg.predict(X_test_scaled)

In [14]:
y_pred_rf = np.expm1(y_pred_log_rf)

In [15]:
print("\n--- Resultados Random Forest Regressor (con Transformación Log) ---")
print(f"R^2 Score: {r2_score(y_test, y_pred_rf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_rf)):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_rf):.4f}")


--- Resultados Random Forest Regressor (con Transformación Log) ---
R^2 Score: 0.7852
RMSE: 5929.8713
MAE: 1335.5792


El Random Forest Regressor, combinado con una transformación logarítmica de la variable objetivo, fue el modelo más efectivo para estimar el monto del pago.

Rendimiento Clave: Se alcanzó un R² Score de 0.79, lo que indica que el modelo puede explicar el 79% de la variabilidad en los montos de pago. El RMSE se redujo a 5,930 NT$, casi la mitad del error del modelo lineal base.

Técnicas Esenciales: El rendimiento superior se logró gracias a:

Modelo no Lineal: El Random Forest fue capaz de capturar las complejas relaciones entre las características y el monto del pago, algo que los modelos lineales no pudieron hacer (su R² fue de solo 0.33).

Transformación de la Variable Objetivo: Aplicar el logaritmo al monto del pago normalizó su distribución y redujo el impacto de los valores atípicos, lo que fue crucial para la precisión del modelo.


Conclusión de Negocio: El modelo es suficientemente preciso para ser utilizado como un sistema de alerta temprana. Si el pago estimado por el modelo es significativamente inferior a la factura del cliente, puede ser un indicador temprano de problemas de liquidez, permitiendo al banco actuar de forma proactiva.